In [1]:
import pandas as pd
import glob
import argparse
import os
import csv
import numpy as np
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
allStats = pd.read_pickle('~/MATH189/project/Data/player_data_cleaned.pkl')

In [3]:
def binData(statsDf, nbins):
    #This returns a new data frame with binned data to run random foresets and decision trees
    binDf = copy.deepcopy(statsDf)
    getMid = lambda interval : interval.mid
    for column in binDf.drop(['ID', 'Year', 'Player', 'Pos', 'Tm', 'MVP'], axis=1, errors='ignore'):
        series = pd.cut(binDf[column], nbins)
        binDf[column] = series.apply(getMid)
    return binDf

In [4]:
binDf = binData(allStats, 5)
x = binDf.iloc[0, 4]
binDf.Age

0        31.000
1        25.800
2        25.800
3        31.000
4        20.587
5        31.000
6        20.587
7        31.000
10       25.800
11       25.800
12       25.800
13       20.587
17       25.800
18       36.200
19       25.800
20       31.000
21       25.800
25       25.800
26       20.587
27       31.000
28       25.800
29       20.587
30       31.000
31       25.800
32       25.800
33       20.587
34       25.800
35       20.587
36       25.800
39       20.587
          ...  
19271    36.200
19272    25.800
19273    25.800
19274    20.587
19275    25.800
19276    20.587
19277    25.800
19278    25.800
19279    31.000
19282    25.800
19285    31.000
19288    31.000
19289    31.000
19290    20.587
19293    25.800
19294    20.587
19295    25.800
19296    20.587
19297    36.200
19298    31.000
19299    25.800
19300    20.587
19301    25.800
19302    31.000
19303    25.800
19304    25.800
19305    25.800
19306    20.587
19307    20.587
19308    20.587
Name: Age, Length: 15817

In [5]:
d = allStats.iloc[:, 4]
d = pd.cut(d, 4)
data = allStats.drop(['ID','Year','Player','Tm', 'MVP', 'Pos'], axis=1, errors='ignore')
for i in data.columns:
    print(i, ' --> Min: ', data[i].min(), '; Max: ', data[i].max(), '; Mean: ', data[i].mean(), '; Std: ', data[i].std())

Age  --> Min:  18.0 ; Max:  44.0 ; Mean:  26.76493646077006 ; Std:  4.042249946917551
G  --> Min:  1.0 ; Max:  85.0 ; Mean:  55.41891635581969 ; Std:  25.06926956341708
MP  --> Min:  1.0 ; Max:  3533.0 ; Mean:  1314.0021495858887 ; Std:  931.2871595355956
PER  --> Min:  -90.6 ; Max:  129.1 ; Mean:  12.78641967503319 ; Std:  5.832420133806965
TS%  --> Min:  0.0 ; Max:  1.136 ; Mean:  0.5074717708794322 ; Std:  0.08874582126479899
FTr  --> Min:  0.0 ; Max:  6.0 ; Mean:  0.32034658911297864 ; Std:  0.2120799728786566
ORB%  --> Min:  0.0 ; Max:  100.0 ; Mean:  6.23763671998483 ; Std:  4.7814134755599405
DRB%  --> Min:  0.0 ; Max:  100.0 ; Mean:  13.826085856989295 ; Std:  6.343673425153647
TRB%  --> Min:  0.0 ; Max:  86.4 ; Mean:  10.033027754947229 ; Std:  4.904221921914214
AST%  --> Min:  0.0 ; Max:  100.0 ; Mean:  13.17311753176964 ; Std:  9.424042067533957
STL%  --> Min:  0.0 ; Max:  24.2 ; Mean:  1.6563001833470354 ; Std:  0.95960307187893
BLK%  --> Min:  0.0 ; Max:  77.8 ; Mean:  1.4

In [6]:
def scaleData(playerDf):
    # Input: cleaned data pandas df
    # Output: scaled X data matrix with unecessary features removed, label vector, as numpy arrays
    X = playerDf.drop(['ID','Year','Player','Tm', 'MVP'], axis=1, errors='ignore')
    y = playerDf.loc[:, allStats.columns == 'MVP'].values
    X_scaled = StandardScaler().fit_transform(X)
    return X_scaled, y

In [7]:
test1, test2 = scaleData(allStats)
test1.shape

/Users/gabrielquiroz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/gabrielquiroz/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(15817, 40)

In [8]:
X = allStats.drop(['ID','Year','Player','Tm', 'MVP'], axis=1, errors='ignore')
players = copy.deepcopy(X)
X = X.values
y = allStats.loc[:, allStats.columns == 'MVP'].values
X_scaled = StandardScaler().fit_transform(X)

In [9]:
def performPca(scaledData, n_components=.95):
    #Inputs: scaled data matrix, number of components for pca (starts off as just trying to get 95% of variance)
    #Outputs: sklearn pca class
    pca = PCA(n_components=n_components)
    principalComp = pca.fit_transform(X_scaled)
    principalDf = pd.DataFrame(data=principalComp)
    return pca, principalDf

In [10]:
pca, pc = performPca(X_scaled)
display(pc)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-13.679662,5.290306,0.648890,-0.901370,0.592958,1.629196,-0.160418,-2.507947,2.853425,2.009300,-1.277146,-1.723364,-1.482238,1.595759,0.747809
1,0.205085,0.587161,-1.375549,0.098404,-0.355155,-1.351785,0.192412,-0.090683,0.025292,-0.552661,-1.342758,0.395732,0.030724,-0.078693,-0.926715
2,-8.149712,2.337311,0.834858,-0.532279,1.970799,0.155667,0.308039,-0.365311,-1.482298,0.601317,-0.889984,-0.470007,-0.482948,-0.355410,-0.261065
3,3.820406,-1.003666,0.185511,-0.561904,0.312587,-0.254352,0.405047,-1.553494,0.241300,0.088577,0.600298,0.149752,-0.126048,0.181851,-0.311370
4,5.312993,0.307579,1.999823,0.468369,0.294925,0.728396,0.395479,0.400324,0.286338,-0.332641,-0.528129,-0.256832,0.283804,-0.399610,0.914875
5,-0.530898,-2.643324,1.412776,0.261588,2.146694,-1.266806,-2.426156,0.338761,0.267618,0.776668,1.024539,0.656860,0.299024,-0.082691,-0.010744
6,4.963699,-1.860959,0.675677,-0.629665,0.708543,-0.539906,0.402406,0.216858,0.593273,-0.399058,-0.853224,-1.237568,0.033254,-0.103968,0.359555
7,3.010677,1.536474,-0.003001,-1.879096,0.368349,0.043913,-2.373994,0.617742,-0.189894,-0.277432,0.021337,-0.128322,0.237091,-0.428722,-0.426776
8,5.640611,-1.702559,2.150482,2.775003,0.915475,1.123424,1.281329,-1.956391,-1.450247,1.169762,-1.042922,-0.793333,1.678878,2.339787,2.872254
9,-2.002090,1.681770,-0.000428,-0.112453,0.104288,-0.906365,0.807912,0.108389,-0.831271,-0.969215,-0.511210,-0.317400,-0.291429,-0.582787,0.787646


In [11]:
pca = PCA(n_components=.95)
pca.fit_transform(X_scaled)

array([[-13.6796623 ,   5.2903056 ,   0.64888958, ...,  -1.48223828,
          1.59575895,   0.74780898],
       [  0.20508468,   0.58716145,  -1.37554886, ...,   0.0307236 ,
         -0.07869293,  -0.92671516],
       [ -8.14971203,   2.33731095,   0.8348583 , ...,  -0.48294832,
         -0.35540993,  -0.26106521],
       ...,
       [  5.19903945,   2.72325824,   0.66335999, ...,   0.21462852,
         -1.31917123,  -0.38740069],
       [  3.09403874,  -0.40377341,   0.07292641, ...,  -0.41229063,
         -0.8736534 ,   0.14769936],
       [  2.11710253,   2.30715203,  -0.98151237, ...,  -0.41663061,
         -0.79561707,  -0.76781455]])

In [12]:

pcaDf = pd.DataFrame(pca.components_,columns=players.columns)
display(pcaDf)
pcaDf.to_csv('~/MATH189/project/Data/pca_95percent.csv')

,Pos,Age,G,MP,PER,TS%,FTr,ORB%,DRB%,TRB%,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,-0.000824,-0.006432,-0.180528,-0.221370,-0.167644,-0.117382,-0.014850,-0.001107,-0.037755,-0.025535,...,-0.212866,-0.168323,-0.197731,-0.193983,-0.161306,-0.191001,-0.131023,-0.211081,-0.193181,-0.223586
1,-0.292689,-0.004040,-0.006738,-0.038910,0.017415,0.018099,0.125912,0.297725,0.330180,0.359635,...,-0.001188,0.222876,0.157111,0.183422,-0.187172,-0.112677,0.234433,-0.055588,0.079437,-0.064957
2,-0.007270,-0.081952,0.038879,0.088356,-0.330627,-0.358678,-0.080603,-0.032193,-0.016964,-0.030028,...,0.107469,0.101182,0.086010,0.093569,0.039279,0.038792,0.059701,0.141336,0.104013,0.102344
3,0.018837,-0.134415,-0.155082,-0.077785,0.197279,0.120222,0.134796,0.196532,0.050832,0.132123,...,0.117521,-0.003287,-0.051173,-0.036968,-0.155376,-0.192089,-0.105332,-0.048860,-0.126195,0.092378
4,-0.036620,-0.180053,-0.172899,-0.108280,0.147920,0.021188,0.320809,0.107329,-0.065478,0.011270,...,0.109464,-0.015670,-0.107572,-0.080591,0.165240,0.084312,-0.075888,0.108343,-0.088937,-0.021782
5,-0.121003,0.065538,-0.273275,-0.075186,-0.033255,-0.172697,0.143977,0.097569,0.248773,0.212193,...,0.035734,-0.155790,0.007486,-0.045943,0.087078,0.008770,-0.026820,-0.031232,-0.224523,0.016756
6,0.235754,-0.471968,-0.070512,-0.048613,0.077263,-0.224186,-0.335108,0.098506,-0.011167,0.040483,...,-0.046582,0.017178,-0.012342,-0.003011,-0.155122,0.078752,0.106415,-0.073658,-0.054224,0.008402
7,-0.005529,-0.665586,0.215071,0.075082,-0.029930,0.250545,0.329886,0.002639,-0.001007,-0.003068,...,0.000628,-0.004380,-0.006818,-0.006221,0.004387,0.028715,0.009966,0.091973,0.165827,-0.009174
8,0.137232,-0.051600,-0.155119,-0.059202,-0.081693,0.029632,0.333663,-0.369581,-0.220853,-0.327678,...,0.138209,-0.081090,-0.066807,-0.073499,-0.148993,-0.123665,0.292606,-0.084429,-0.110562,0.043527
9,-0.328399,-0.051984,-0.015115,-0.028031,0.191105,0.048749,-0.409986,-0.105533,0.011197,-0.044282,...,-0.064847,-0.141970,-0.048150,-0.080420,0.180368,-0.108073,0.205155,0.110354,-0.020187,0.009077


In [13]:
absDf = pcaDf.abs()
maxCorr = absDf.max(axis=1)
for i in range(len(maxCorr)):
    print(absDf.columns[maxCorr[i] == absDf.iloc[i]], maxCorr[i])

Index(['PTS'], dtype='object') 0.2235855544165491
Index(['TRB%'], dtype='object') 0.3596350838316012
Index(['WS/48'], dtype='object') 0.4445245290693437
Index(['DBPM'], dtype='object') 0.4534701650388007
Index(['3P'], dtype='object') 0.3885388502565108
Index(['3P'], dtype='object') 0.3764878951344636
Index(['Age'], dtype='object') 0.471967973432145
Index(['Age'], dtype='object') 0.6655856446302418
Index(['BLK%'], dtype='object') 0.40856783802547936
Index(['BLK%'], dtype='object') 0.44204849268431423
Index(['Age'], dtype='object') 0.4845995180307101
Index(['TOV%'], dtype='object') 0.5689854361814668
Index(['ORB%'], dtype='object') 0.5503408448893816
Index(['DRB%'], dtype='object') 0.44854522478314296
Index(['Pos'], dtype='object') 0.6793484015899712
